In [1]:
from torchvision import datasets
import torch
data_folder = ''
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True)

In [3]:
tr_images = fmnist.data
tr_targets = fmnist.targets

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
R, C = len(tr_targets.unique()), 10  # Define o número de linhas (R) como o número de classes únicas em `tr_targets` e o número de colunas (C) como 10.
fig, ax = plt.subplots(R, C, figsize=(10, 10))  # Cria uma grade de subplots com R linhas e C colunas, com tamanho total de figura 10x10.

for label_class, plot_row in enumerate(ax):  # Itera sobre cada classe de rótulo e suas respectivas linhas de subplots.
    label_x_rows = np.where(tr_targets == label_class)[0]  # Encontra os índices das imagens pertencentes à classe atual.

    for plot_cell in plot_row:  # Itera sobre as células (colunas) da linha atual.
        plot_cell.grid(False); plot_cell.axis('off')  # Remove as grades e os eixos de cada subplot.
        ix = np.random.choice(label_x_rows)  # Seleciona aleatoriamente um índice da classe atual.
        x, y = tr_images[ix], tr_targets[ix]  # Obtém a imagem `x` e seu rótulo `y` a partir do índice selecionado.
        plot_cell.imshow(x, cmap='gray')  # Exibe a imagem em escala de cinza na célula correspondente.

plt.tight_layout()  # Ajusta automaticamente o espaçamento entre os subplots para evitar sobreposição.


In [7]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from torchvision import datasets

class FMNISTDataset(Dataset):  # Criação de uma classe personalizada de dataset para Fashion MNIST, herdando de `Dataset`.
    def __init__(self, x, y):
        x = x.float()  # Converte os dados `x` para o tipo `float`.
        # Estamos achatando cada imagem, altura = largura = 28.
        # -1 significa que as outras dimensões serão ajustadas automaticamente com base no número de elementos.
        x = x.view(-1, 28*28)  # Redimensiona cada imagem 28x28 para um vetor unidimensional de tamanho 784.
        self.x, self.y = x, y  # Salva os dados de entrada `x` e os rótulos `y`.

    def __getitem__(self, idx):
        x, y = self.x[idx], self.y[idx]  # Obtém a entrada `x` e o rótulo `y` correspondentes ao índice fornecido.
        return x.to(device), y.to(device)  # Transfere os dados para o dispositivo (CPU ou GPU) e os retorna.

    def __len__(self):
        return len(self.x)  # Retorna o número total de amostras no dataset.


In [9]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True)
    return trn_dl

In [11]:
#alterei o modelo reduzindo a quantidade de unidades na camada

from torch.optim import SGD
import torch.nn as nn

def get_model():
    # Modifiquei a estrutura do modelo, adicionando uma camada intermediária
    model = nn.Sequential(
        nn.Linear(28 * 28, 512),        # Reduzi a quantidade de unidades da camada
        nn.ReLU(),
        nn.Dropout(0.3),                 # Adicionei dropout para regularização
        nn.Linear(512, 10)               # Camada de saída
    ).to(device)
    
    # Mantive a função de perda CrossEntropy
    loss_fn = nn.CrossEntropyLoss()
    
    # Modifiquei o otimizador, agora com SGD e Momentum
    optimizer = SGD(model.parameters(), lr=1e-2, momentum=0.9)  # Usando momentum para melhorar a convergência
    
    return model, loss_fn, optimizer


In [13]:
@torch.no_grad()
def accuracy_fn(x,y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [15]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()  # Coloca o modelo em modo de treinamento (habilita dropout, batch normalization, etc.).
    prediction = model(x)  # Faz a previsão passando os dados de entrada `x` pelo modelo.
    batch_loss = loss_fn(prediction, y)  # Calcula a perda (loss) comparando as previsões com os rótulos reais `y`.
    batch_loss.backward()  # Calcula os gradientes através da retropropagação (backpropagation).
    optimizer.step()  # Atualiza os pesos do modelo com base nos gradientes calculados.
    # Limpa a memória dos gradientes para a próxima iteração.
    optimizer.zero_grad()  
    return batch_loss.item()  # Retorna o valor da perda como um número escalar.


In [17]:
trn_dl = get_data()
model, loss_fn, optimizer = get_model()

In [19]:
losses, accuracies = [], []
for epoch in range(5):
    print(epoch)
    epoch_losses, epoch_accuracies = [], []
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        epoch_losses.append(batch_loss)
    epoch_loss = np.array(epoch_losses).mean()
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        is_correct = accuracy_fn(x, y, model)
        epoch_accuracies.extend(is_correct)
    epoch_accuracy = np.mean(epoch_accuracies)
    losses.append(epoch_loss)
    accuracies.append(epoch_accuracy)

0
1
2
3
4


In [20]:
epochs = np.arange(5) + 1  # Cria um array com os valores das épocas (1 a 5)
plt.figure(figsize=(20, 5))  # Define o tamanho da figura

# Gráfico da perda
plt.subplot(121)
plt.title('Loss value over increasing epochs')  # Título do gráfico
plt.plot(epochs, losses, label='Training Loss')  # Plota os valores da perda
plt.legend()  # Exibe a legenda

# Gráfico da acurácia
plt.subplot(122)
plt.title('Accuracy value over increasing epochs')  # Título do gráfico
plt.plot(epochs, accuracies, label='Training Accuracy')  # Plota os valores da acurácia

# Define ticks fixos antes de personalizar os rótulos do eixo Y
yticks = plt.gca().get_yticks()  # Obtém os ticks atuais
plt.gca().set_yticks(yticks)  # Define os ticks explicitamente
plt.gca().set_yticklabels(['{:.0f}%'.format(x * 100) for x in yticks])  # Converte os rótulos para porcentagem
plt.legend()  # Exibe a legenda
